In [63]:
# Load environment variables from .env file
from dotenv import load_dotenv

# Load .env file
load_dotenv()

import os
from vexa_client import VexaClient

import os
TOKEN = os.getenv('ADMIN_API_TOKEN')

In [64]:
url = "http://localhost:8056"

In [65]:
admin_client = VexaClient(
    base_url=url,
    admin_key=TOKEN  # From docker-compose.yml
)

In [66]:
import random

In [67]:
new_user = admin_client.create_user(email=f"{random.randint(1, 1000000)}@example.com", name="test")



DEBUG: Making POST request to http://localhost:8056/admin/users
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: {'email': '104822@example.com', 'name': 'test'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sat, 19 Apr 2025 15:13:44 GMT, Sat, 19 Apr 2025 15:13:45 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '111', 'content-type': 'application/json'}
DEBUG: Response content: {"email":"104822@example.com","name":"test","image_url":null,"id":59,"created_at":"2025-04-19T15:13:45.582115"}...


In [68]:

user_id = new_user['id']

In [69]:
user_id

59

In [70]:
token_info = admin_client.create_token(user_id=user_id)
user_api_key = token_info['token']


DEBUG: Making POST request to http://localhost:8056/admin/users/59/tokens
DEBUG: Headers: {'Content-Type': 'application/json', 'X-Admin-API-Key': 'KJnl2jjk&'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sat, 19 Apr 2025 15:13:44 GMT, Sat, 19 Apr 2025 15:13:45 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '115', 'content-type': 'application/json'}
DEBUG: Response content: {"user_id":59,"id":59,"token":"zNwnRF6fTJto1Ku2wVunPdKByDTXBXdjc3XO4zAw","created_at":"2025-04-19T15:13:45.632111"}...


In [71]:
#user_api_key = 'FQUV46o4ychNZOKMEDDEkCJ7hL82rgOomHTmKMgH'

In [72]:
client = VexaClient(
    base_url=url,
    api_key=user_api_key
)

In [73]:
meeting_url="https://meet.google.com/qcm-yvzc-izx"
native_meeting_id = meeting_url.split("/")[-1]  # "xyz-abcd-123"

In [89]:
meeting_info = client.request_bot(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    bot_name="MyBot",
  #  language='es',
    task = 'transcribe'
)
meeting_id = meeting_info['id']


DEBUG: Making POST request to http://localhost:8056/bots
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'zNwnRF6fTJto1Ku2wVunPdKByDTXBXdjc3XO4zAw'}
DEBUG: Params: None
DEBUG: JSON data: {'platform': 'google_meet', 'native_meeting_id': 'qcm-yvzc-izx', 'bot_name': 'MyBot', 'task': 'transcribe'}
DEBUG: Response status: 201
DEBUG: Response headers: {'date': 'Sat, 19 Apr 2025 15:16:47 GMT, Sat, 19 Apr 2025 15:16:47 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '393', 'content-type': 'application/json'}
DEBUG: Response content: {"id":76,"user_id":59,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","bot_container_id":"1583b8857f68318e7a8c03c5a3e06125dfb3d6b7f17e1d7676e7369a313ba5a3","start_time":"2025-04-19T15:16:48.502997","end_time":null,"created_at":"2025-04-19T15:16:48.306735","updated_at":"2025-04-19T15:16:48.314247"}...


In [75]:
meeting_id

74

In [90]:
client.update_bot_config(
    platform="google_meet",
    native_meeting_id=native_meeting_id,
    language='en',
    task = 'transcribe'
)


DEBUG: Making PUT request to http://localhost:8056/bots/google_meet/qcm-yvzc-izx/config
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'zNwnRF6fTJto1Ku2wVunPdKByDTXBXdjc3XO4zAw'}
DEBUG: Params: None
DEBUG: JSON data: {'language': 'en', 'task': 'transcribe'}
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Sat, 19 Apr 2025 15:18:51 GMT, Sat, 19 Apr 2025 15:18:50 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '67', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Reconfiguration request accepted and sent to the bot."}...


{'message': 'Reconfiguration request accepted and sent to the bot.'}

In [88]:
client.stop_bot(platform="google_meet",native_meeting_id=native_meeting_id)


DEBUG: Making DELETE request to http://localhost:8056/bots/google_meet/qcm-yvzc-izx
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'zNwnRF6fTJto1Ku2wVunPdKByDTXBXdjc3XO4zAw'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 202
DEBUG: Response headers: {'date': 'Sat, 19 Apr 2025 15:16:38 GMT, Sat, 19 Apr 2025 15:16:37 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '59', 'content-type': 'application/json'}
DEBUG: Response content: {"message":"Stop request accepted and is being processed."}...


{'message': 'Stop request accepted and is being processed.'}

In [14]:
meetings = client.get_meetings()


DEBUG: Making GET request to http://localhost:8056/meetings
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'z1X5GaBMVR9B88CJH4f7sr2epjJnoR25AGdEfQFP'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Fri, 18 Apr 2025 18:34:12 GMT, Fri, 18 Apr 2025 18:34:12 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '408', 'content-type': 'application/json'}
DEBUG: Response content: {"meetings":[{"id":64,"user_id":54,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","bot_container_id":"a50ebcbae367f255fed6c8506161f60382fbb90fa3c953fee7d843bd60fd4955","start_time":"2025-04-18T18:33:38.770287","end_time":null,"created_at":"2025-04-18T18:33:38.536475","updated_at":"2025-04-18T18:33:38.542903"}]}...


In [15]:
meeting_id

64

In [93]:
import pandas as pd
pd.options.display.max_colwidth = 1000
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', '{:20.2f}'.format)
pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
transcript = client.get_transcript(native_meeting_id=native_meeting_id,platform='google_meet')
df = pd.DataFrame(transcript['segments'])#.sort_values(['created_at','start'])#.drop_duplicates('start',keep='last')
df.sort_values('absolute_start_time').tail(5)


DEBUG: Making GET request to http://localhost:8056/transcripts/google_meet/qcm-yvzc-izx
DEBUG: Headers: {'Content-Type': 'application/json', 'X-API-Key': 'zNwnRF6fTJto1Ku2wVunPdKByDTXBXdjc3XO4zAw'}
DEBUG: Params: None
DEBUG: JSON data: None
DEBUG: Response status: 200
DEBUG: Response headers: {'date': 'Sat, 19 Apr 2025 15:18:59 GMT, Sat, 19 Apr 2025 15:18:59 GMT', 'server': 'uvicorn, uvicorn', 'content-length': '2577', 'content-type': 'application/json'}
DEBUG: Response content: {"id":76,"platform":"google_meet","native_meeting_id":"qcm-yvzc-izx","constructed_meeting_url":"https://meet.google.com/qcm-yvzc-izx","status":"active","start_time":"2025-04-19T15:16:48.502997","end_time":null,"segments":[{"start":0.0,"end":15.48,"text":" Вот, так, сейчас мы детектируем, так, одну минуту, хорошо, поработаем, идем.","language":null,"created_at":null,"speaker":null,"absolute_start_time":"2025-04-19T15:17:06.472579+00:00","absolute_end_time":"2025-04-19T15:17:21.952579+00:00"},{"st...


,start,end,text,language,created_at,speaker,absolute_start_time,absolute_end_time
4,51.84,53.68,Пишите мой номер.,None,2025-04-19T15:18:32.934416,None,2025-04-19T15:17:58.314579+00:00,2025-04-19T15:18:00.154579+00:00
5,92.61,94.61,Окей...,None,None,None,2025-04-19T15:18:39.084579+00:00,2025-04-19T15:18:41.084579+00:00
6,94.61,97.61,Окей...,None,None,None,2025-04-19T15:18:41.084579+00:00,2025-04-19T15:18:44.084579+00:00
7,96.55,104.95,"Окей, так, ну что, можно теперь поменять язык на английский?",None,None,None,2025-04-19T15:18:43.024579+00:00,2025-04-19T15:18:51.421579+00:00
8,0.30,5.30,"I changed my language to English, and now I'm watching you.",None,None,None,2025-04-19T15:18:54.102292+00:00,2025-04-19T15:18:59.102292+00:00


In [20]:
df

""
